In [29]:
import torch
from torch.autograd import Variable
import numpy
import ctypes

In [24]:
w1 = Variable(torch.ones(2, 2)*2, requires_grad = True)
w2 = Variable(torch.ones(2, 2)*3, requires_grad = True)
loss = torch.sum(w1 + w2)
print(loss.data)
print(loss.grad_fn)
loss.backward()
print(w1.grad)
print(w2.grad)

tensor(20.)
tensor([[1., 1.],
        [1., 1.]])
tensor([[1., 1.],
        [1., 1.]])


In [26]:
loss.data_ptr()

93842032237696

In [46]:
arr = w1.cpu().detach().numpy()
n = 2
m = 2
# a = (ctypes.POINTER(ctypes.c_float) * n)()
# for i in range(m):
#     a[i] = ctypes.cast((ctypes.c_float * n)(arr[i]), ctypes.POINTER(ctypes.c_float))
# b = (ctypes.c_float * n)(arr[i])

TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
w1 = Variable(torch.ones(2, 2)*2, requires_grad = True)
w2 = Variable(torch.ones(2, 2)*3, requires_grad = True)
# loss = sum(w1,w2)
# loss.backward()
# w1.grad